<a href="https://colab.research.google.com/github/GHROTH-L/PSFD_Department-Classification-/blob/main/%E6%95%B4%E7%90%86%E5%AD%B8%E7%B3%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 科系整理

##先整理大學的科系與對應

In [ ]:
from google.colab import files
import pandas as pd

# 選擇上傳 Excel 檔案
uploaded = files.upload()

# 取得檔案名稱
file_name = list(uploaded.keys())[0]

# 讀取 Excel
df = pd.read_excel(file_name)

# 顯示前幾筆資料
df.head()

In [ ]:
expanded_data = []
for index, row in df.iterrows():
    if pd.notna(row["discrib"]):  # 確保第五欄有值
        split_values = row["discrib"].split("、")  # 依據頓號切割
        for value in split_values:
            expanded_data.append([row["mainid"], row["midid"], row["lastid"], value, row["eduid"]])  # 重新組合成新行
    else:
        expanded_data.append([row["mainid"], row["midid"], row["lastid"], row["lastname"], row["eduid"]])  # 沒有第五欄的保留原始

# 轉換為新的 DataFrame
expanded_df = pd.DataFrame(expanded_data, columns=["mainid", "midid", "lastid	", "lastname", "eduid"])

In [ ]:
expanded_df.head(10)

In [ ]:
# 假設 df 是你的 DataFrame
output_file = "output.xlsx"  # 輸出的檔案名稱

# 將 DataFrame 儲存為 Excel
expanded_df.to_excel(output_file, index=False)

# 讓 Colab 下載 Excel 檔案
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##ckip 與 jeiba

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
! pip install -U ckip-transformers
from ckip_transformers import __version__
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
from google.colab import files
import jieba

In [ ]:
# 文本預處理，這裡可以使用 jieba 進行中文分詞
def jieba_text(text):
    # 使用 jieba 分詞
    words = jieba.cut(text)
    return ' '.join(words)

In [ ]:
# Show version
print(__version__)

# Initialize drivers
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(model="albert-base", device=-1)
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(model="albert-base", device=-1)
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(model="albert-base", device=-1)
print("Initializing drivers ... all done")
print()

#model 有其它的可以選，如 "bert-base"
#device=0 是使用 GPU， device=-1 是使用 CPU，不指定也可以。
def clean(sentence_ws, sentence_pos):
  short_with_pos = []
  short_sentence = []
  stop_pos = set(['Nep', 'Nh', 'Nb',]) # 這 3 種詞性不保留
  for word_ws, word_pos in zip(sentence_ws, sentence_pos):
    # 只留名詞和動詞
    is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
    # 去掉名詞裡的某些詞性
    is_not_stop_pos = word_pos not in stop_pos
    # 只剩一個字的詞也不留
    is_not_one_charactor = not (len(word_ws) == 1)
    # 組成串列
    if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
      short_with_pos.append(f"{word_ws}({word_pos})")
      short_sentence.append(f"{word_ws}")
  return (" ".join(short_sentence), " ".join(short_with_pos))


In [ ]:
uploaded = files.upload() #上傳資料
# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
data = pd.read_csv(filename)

In [ ]:
#確認是否有亂碼
with open(filename, "rb") as f:
    for i, line in enumerate(f):
        try:
            line.decode("utf-8")
        except UnicodeDecodeError as e:
            print(f"錯誤發生在第 {i+1} 行，錯誤訊息: {e}")
            break
print(data.dtypes)
print(data.head(10))

In [ ]:
data.dropna(subset=['mainid'], inplace=True)
data['mainid'] = data['mainid'].astype(int)

In [ ]:
# 儲存分詞結果與新的句子
data["result"] = ""
data["n_lastname"] = ""

# 進行每個部份的分析
for index, row in data.iterrows():
    description = row["lastname"]
    # 分詞
    ws_result = ws_driver([description])
    # 詞性標註
    pos_result = pos_driver(ws_result)

    # 清洗數據
    cleaned_sentence, cleaned_with_pos = clean(ws_result[0], pos_result[0])

    # 保存结果
    data.at[index, "result"] = cleaned_with_pos
    data.at[index, "n_lastname"] = cleaned_sentence

In [ ]:
data['jeiba_lastname'] = data['lastname'].apply(jieba_text)
data['n_lastname'] = data['n_lastname'].fillna(data['jeiba_lastname'])
'''
nan_indices = data[data['n_lastname'].isna()].index
print(nan_indices)
nan_rows_details = data.loc[[425, 2172]]
print(nan_rows_details)

不分系 跟基督教學系會被nan

'''
print(data.head(10))


In [ ]:
nan_indices = data[data['n_lastname'].isna()].index
print(nan_indices)
nan_rows_details = data.loc[[425, 2172]]
nan_rows_details

In [ ]:
output_filename = "2022科系分類_奕嘉_已經分詞.csv"
data.to_csv(output_filename, index=False, encoding='utf-8-sig')
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##分類測試

### 隨機森林

In [ ]:
import pandas as pd
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from google.colab import files

from imblearn.over_sampling import SMOTE


In [ ]:
uploaded = files.upload() #上傳資料
# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
data = pd.read_csv(filename)

In [ ]:
nan_indices = data[data['n_lastname'].isna()].index
print(nan_indices)
nan_rows_details = data.loc[[425, 2172]]
print(nan_rows_details)


In [ ]:
nan_rows_details = data.loc[[425, 2172]]
print(nan_rows_details)

In [ ]:
# 特徵向量化，使用 Tfidf 向量化
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['n_lastname'])

# 職業編號當作目標變量
y = data['mainid']

# 分割數據為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# 訓練模型
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)

# 預測和評估
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
#準確率不高，所以採用smote 後再進行

# 檢查類別分部
data_df = pd.DataFrame({'lastname': data['n_lastname'], 'label': data['mainid']})
label_counts = data_df['label'].value_counts()

# 查看是否有少於5筆的樣本
rare_labels = label_counts[label_counts < 6].index
print(f"稀有類別: {rare_labels}")


In [ ]:
# 處理不均衡數據 (使用 SMOTE)
X = data_df['lastname'].tolist()
y = data_df['label'].tolist()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 將文本數據轉換為數值特徵
vectorizer = TfidfVectorizer(max_features=2500, min_df=1, max_df=0.9)  # 取前 2000 個高頻詞
X_vectorized = vectorizer.fit_transform(X)


# 使用 SMOTE 平衡類別分佈
smote = SMOTE(sampling_strategy='not minority', random_state=42, k_neighbors=5) #複製not minority
X_resampled, y_resampled = smote.fit_resample(X_vectorized, y_encoded)

# 分割數據集
train_texts, val_texts, train_labels, val_labels = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=69, stratify=y_resampled
)

In [ ]:
# 訓練模型：這裡用訓練特徵和訓練標籤
model = RandomForestClassifier(n_estimators=1000)
model.fit(train_texts, train_labels)

# 預測和評估：用驗證特徵進行預測，並將預測結果與驗證標籤比較
y_pred = model.predict(val_texts)
print(classification_report(val_labels, y_pred))

### 對答案


In [ ]:
uploaded = files.upload() #上傳資料
# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
data2 = pd.read_csv(filename)

In [ ]:
def predict_and_evaluate(new_data, model):
    # 取得預測結果，注意這裡假設模型接受 ln_lastname 欄位作為輸入
    # 將文本數據轉換為數值特徵
    X_new = vectorizer.transform(new_data['n_lastname'])

    predicted_ids = model.predict(X_new)

    # 使用相同的 label_encoder 將編碼數字轉回原始標籤
    predicted_labels = label_encoder.inverse_transform(predicted_ids)

    # 將預測結果加入 DataFrame 中
    new_data['predicted_label'] = predicted_labels

    # 根據預測結果與原始 mainid 進行比較，產生 code 欄位 (1 表示預測正確，0 表示錯誤)
    new_data['code'] = (new_data['predicted_label'] == new_data['mainid']).astype(int)

    # 計算準確率
    accuracy = new_data['code'].mean()
    print("準確率：", accuracy)
    return new_data, accuracy

In [ ]:
data2, acc = predict_and_evaluate(data, model)

In [ ]:
#輸出結果
output_filename = "預測結果比對.csv"
data2.to_csv(output_filename, index=False, encoding='utf-8-sig')
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###儲存模型

In [ ]:
import joblib
from google.colab import files

# 儲存模型
joblib.dump(model, 'model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')
joblib.dump(label_encoder, 'label_encoder.joblib')

# 下載模型檔案到本地
files.download('model.joblib')
files.download('vectorizer.joblib')
files.download('label_encoder.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 上傳模型進行使用

In [ ]:
import joblib

# 載入模型
model = joblib.load('model.joblib')

# 載入 TF-IDF vectorizer
vectorizer = joblib.load('vectorizer.joblib')

# 載入 LabelEncoder
label_encoder = joblib.load('label_encoder.joblib')